## 제목 : Pytorch Classification RNN
## 작성자 : 김수
## 작성 일자 : 2022.02.23

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torch.nn.init as init
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from torch.autograd import Variable
torch.manual_seed(777)
num_epoch = 1000
Learning_Rate = 0.01
day = 48
seq_length = 3 * day
predict_day = 2 * day

In [2]:
training_data = np.loadtxt("G:/공유 드라이브/연구실 폴더/연구실 학습/2021년 겨울방학/Machine Learning/8차/Solar_TRAIN.csv",delimiter=",",skiprows=1)
test_data = np.loadtxt("G:/공유 드라이브/연구실 폴더/연구실 학습/2021년 겨울방학/Machine Learning/8차/Solar_TEST.csv",delimiter=",",skiprows=1)

train = training_data[:,3:]
test = test_data[:,3:]

In [3]:
print(train.shape)
print(test.shape)

(48240, 6)
(3360, 6)


In [4]:
def build_dataset(training_data, seq_length,predict_day,day):
    x = []
    y = []
    for i in range(0,training_data.shape[0] - seq_length - predict_day + 1,day):
        x.append(training_data[i:i+seq_length,:])
        y.append(training_data[i+seq_length:i+seq_length+predict_day,-1])
    return torch.tensor(x,dtype=torch.float32),torch.tensor(y,dtype=torch.float32)

def build_testset(test_data, seq_length,day):
    x = []
    for i in range(day*7,day*7+test_data.shape[0],day*7):
        x.append(test_data[i-seq_length:i,:])
    return torch.tensor(x,dtype=torch.float32)

In [5]:
TrainX, TrainY = build_dataset(train, seq_length, predict_day, day)
TestX = build_testset(test, seq_length, day)
print(TrainX.shape)
print(TrainY.shape)
print(TestX.shape)

torch.Size([1001, 144, 6])
torch.Size([1001, 96])
torch.Size([10, 144, 6])


C:\Users\user\AppData\Local\Temp/ipykernel_15148/46365531.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  return torch.tensor(x,dtype=torch.float32),torch.tensor(y,dtype=torch.float32)


In [6]:
# cut_line = int(training_data.shape[0]*0.7)
# x = training_data[:cut_line,:12]
# y = training_data[:cut_line,-4:]

# test_x = test_data[:,:12]

# X = torch.tensor(x,dtype=torch.float32)
# Y = torch.reshape(torch.tensor(y,dtype=torch.float32),[y.shape[0],y.shape[1]])
# Y_not_one_hot_encoding = torch.argmax(Y,dim=1)

# Val_X = torch.tensor(val_x,dtype=torch.float32)
# Val_Y = torch.reshape(torch.tensor(val_y,dtype=torch.float32),[val_y.shape[0],val_y.shape[1]])
# Val_Y_not_one_hot_encoding = torch.argmax(Val_Y,dim=1)

# TEST = torch.tensor(test_x,dtype=torch.float32)

# print(X.shape)
# print(Y.shape)
# print(Val_X.shape)
# print(Val_Y.shape)
# print(TEST.shape)

In [7]:
class LSTM(nn.Module): 
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length): 
        super(LSTM, self).__init__() 
        self.num_classes = num_classes 
        #number of classes 
        self.num_layers = num_layers 
        #number of layers 
        self.input_size = input_size 
        #input size 
        self.hidden_size = hidden_size 
        #hidden state 
        self.seq_length = seq_length 
        #sequence length 
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers) 
        #lstm 
        self.fc_1 = nn.Linear(hidden_size, 128) 
        #fully connected 1 
        self.fc = nn.Linear(128, num_classes) 
        #fully connected last layer 
        self.relu = nn.ReLU() 
    def forward(self,x): 
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        #hidden state 
        
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        #internal state # Propagate input through LSTM output, 
        (hn, cn) = self.lstm(x, (h_0, c_0)) 
        #lstm with input, hidden, and internal state 
        #hn = hn.reshape(-1, self.hidden_size) 
        #reshaping the data for Dense layer next
        out = self.relu(hn) 
        out1 = self.fc_1(out) 
        #first Dense 
        out2 = self.relu(out1) 
        #relu 
        out3 = self.fc(out2) 
        #Final Output 
        return out3


In [8]:
num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = TrainX.shape[2] #number of features
hidden_size = 10 #number of features in hidden state
num_layers = 3 #number of stacked lstm layers

num_classes = 96 #number of output classes 
lstm_model = LSTM(num_classes, input_size, hidden_size, num_layers, TrainX.shape[1])
print(lstm_model)

LSTM(
  (lstm): LSTM(6, 10, num_layers=3)
  (fc_1): Linear(in_features=10, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=96, bias=True)
  (relu): ReLU()
)


In [9]:
Loss_fun = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=learning_rate)  # adam optimizer

Loss = np.zeros((num_epoch,1))
Val_Loss = np.zeros((num_epoch,1))

#학습
for i in range(num_epoch):
    optimizer.zero_grad()
    output = lstm_model(TrainX)
    loss = Loss_fun(output,TrainY)
    loss.backward()
    optimizer.step()
    Loss[i,0] = loss.data
    if i % 10 == 0:
        print(i,"Loss : ",loss.data)
        

RuntimeError: Expected hidden[0] size (3, 144, 10), got [3, 1001, 10]

In [ ]:
Predict = lstm_model(TestX)
print(Predict.shape)


In [ ]:
predict_ex = Predict.detach()
plt.plot(predict_ex[0,:],label = "Predict Example")
plt.grid()
plt.legend()

In [ ]:
np.savetxt("G:/내 드라이브/연구실/PyTorch/7주차/Test_Predict.csv",predict_ex,delimiter=",",fmt='%d')